In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import re
import time

In [2]:
def get_data(sheet_name):
    file_path = "CLSP+ST-instances Data-R.xlsx"  
    numbers = re.findall(r'\d+', sheet_name)
    number_items = int(numbers[0])
    number_capacity = int(numbers[1])
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)

    # Identify matrix boundaries manually or by searching keywords
    demand_start = df[df[0] == "Demand Forecast:"].index[0] + 1
    production_start = df[df[0] == "Production Cost"].index[0] + 1
    holding_start = df[df[0] == "Holding Cost"].index[0] + 1
    setup_start = df[df[0] == "Setup Cost"].index[0] + 1
    capacity_start = df[df[0] == "Capacity"].index[0] + 1

    # Extract matrices and remove first row and first column
    demand_forecast = df.iloc[demand_start+1:production_start-2, 1:].reset_index(drop=True)
    production_cost = df.iloc[production_start+1:holding_start-2, 1:].reset_index(drop=True)
    holding_cost = df.iloc[holding_start+1:setup_start-2, 1:].reset_index(drop=True)
    setup_cost = df.iloc[setup_start+1:setup_start+number_items+1, 1:].reset_index(drop=True)
    capacity = df.iloc[capacity_start:capacity_start+number_capacity+1, 1:2].reset_index(drop=True)

    UnitsOfCapacity_SupTime = df.iloc[setup_start+number_items+2:capacity_start-2, 1:3].reset_index(drop=True)
    UnitsOfCapacity_SupTime.columns = UnitsOfCapacity_SupTime.iloc[0]
    UnitsOfCapacity_SupTime = UnitsOfCapacity_SupTime[1:].reset_index(drop=True)
    UnitsOfCapacity = UnitsOfCapacity_SupTime['UnitsOfCapacity']
    SetupTime = UnitsOfCapacity_SupTime['SetupTime']

    # Convert to numeric values
    demand_forecast = demand_forecast.apply(pd.to_numeric, errors='coerce')
    production_cost = production_cost.apply(pd.to_numeric, errors='coerce')
    holding_cost = holding_cost.apply(pd.to_numeric, errors='coerce')
    setup_cost = setup_cost.apply(pd.to_numeric, errors='coerce')

    data = {}
    data['Demand Forecast'] = demand_forecast
    data['Production Cost'] = production_cost
    data['Holding Cost'] = holding_cost
    data['Setup Cost'] = setup_cost
    data['UnitsOfCapacity'] = UnitsOfCapacity
    data['SetupTime'] = SetupTime
    data['Capacity'] = capacity
    return data

In [3]:
# data = get_data('Data-20-12 (2)')
# data.keys()

def defineData(sheet):

    #print('Defining Data -----')
    data = get_data(sheet)
    
    N, T = data['Demand Forecast'].shape
    
    #define data + add padding for dummy periods and products
    demand = np.pad(np.array(data['Demand Forecast']), ((1,0), (1,0))) #All demand demand[producttype, time]
    c = np.pad(np.array(data['Production Cost']), ((1,0), (1,0))) #Production Cost [producttype, time]
    h = np.pad(np.array(data['Holding Cost']), ((1,0), (1,0))) #holding Cost ||
    f = np.pad(np.array(data['Setup Cost']), ((1,0), (1,0))) #Setup cost ||
    r = np.pad(np.array(data['UnitsOfCapacity']), (1, 0)) #Capacity units (only in N)
    kappa = np.pad(np.array(data['Capacity']).flatten(), (1,0)) #Total available Capacity (only in T)
    tau = np.pad(np.array(data['SetupTime']), (1,0)) #Setup Time (Only in N)
    
    #Initial Inventory Cost to force feasibility for the RMP
    initialCost = 1e4

    return N, T, demand, c, h, f, r, kappa, tau, initialCost


In [4]:
#Trivial plans
def initial_columns():
    columns = {i: [] for i in range(N+1)}

    for i in range(N+1):
        plan = {
            'x': [0] * (T+1),
            'y': [0] * (T+1),
            's': [sum(demand[i, :])] + ([0] * T),
            'cost': sum(demand[i, :])*initialCost,
            'cap': [0] * (T+1)       
        }
        columns[i].append(plan)
    return columns

In [5]:
#print(initial_columns())

In [6]:
# #Trivial plans
# def initial_columns():
#     columns = {i: [] for i in range(N+1)}

#     for i in range(N+1):
#         plan = {
#             'x': [demand[i,t] for t in range(T+1)],
#             'y': [1 if demand[i, t] > 0 else 0 for t in range(T+1)],
#             's': [0] * (T+1),
#             'cost': sum(f[i,t]*int(demand[i, t] > 0) + c[i, t]*demand[i,t] for t in range(1, T+1)),
#             'cap': [r[i]*demand[i, t] + tau[i]*int(demand[i,t]>0) for t in range(T+1)]        
#         }
#         columns[i].append(plan)
#     return columns

In [7]:
#TEST
#print(initial_columns())

In [29]:
# ------------------------------
# STEP 3: Restricted Master Problem (RMP)
# ------------------------------
def solve_rmp(columns):
    
    #print("--------Solving RMP----------")
    model = gp.Model("RMP")
    model.setParam('OutputFlag', 0)

    #Define the weights
    z = {}
    for i in range(1, N+1): #iterate over each product and plan -> These are the weights without dummy 
        for k, _ in enumerate(columns[i]):
            z[i, k] = model.addVar(vtype=GRB.CONTINUOUS, name=f"z_{i}_{k}")

    model.update()
    
    #RMP Objective
    model.setObjective(gp.quicksum(plan['cost'] * z[i, k] for i in range(1, N+1) for k, plan in enumerate(columns[i])), GRB.MINIMIZE)

   
    #Capacity respected in each period
    for t in range(1, T+1):
        constr = model.addConstr(
            gp.quicksum(plan['cap'][t]*z[i, k] for i in range(1, N+1) for k, plan in enumerate(columns[i])) <= kappa[t],
            name=f"cap_{t}"
        )
    
    for i in range(1, N+1): #Exactly one schedule per product
        model.addConstr(gp.quicksum(z[i, k] for k in range(len(columns[i]))) == 1, name=f"item_{i}")

    
    model.update()
    model.optimize()

    duals = model.getAttr(GRB.Attr.Pi)
    #print(len(duals), duals)
    #for c in model.getConstrs():
        #print(f"{c.ConstrName} dual: {c.Pi}")

    model.update()

    z_values = {(i, k): z[i, k].X for i in range(1, N+1) for k, _ in enumerate(columns[i])}

    return model.ObjVal, duals, z_values

In [31]:
# ------------------------------
# STEP 4: Subproblem (Pricing)
# ------------------------------
def solve_subproblem(i, T, demand_i, f_i, c_i, h_i, r_i, tau_i, duals):

    #print(f"--------Solving Subproblem product {i}----------")
    
    model = gp.Model(f"Subproblem_{i}")
    model.setParam('OutputFlag', 0)

    x = model.addVars(T+1, vtype=GRB.INTEGER, name="x") #With dummy period
    s = model.addVars(T+1, vtype=GRB.INTEGER, name="s")
    y = model.addVars(T+1, vtype=GRB.BINARY, name="y")

    dual_capacity = np.pad(duals[:T], (1,0)) #dummy period
    dual_convexity = np.pad(duals[T:], (1,0)) #dummy product
    #print(dual_capacity)
    #print(dual_convexity)


    #Minimize the reduced cost, if it is negative, it can improve the reduced master problem.
    #Cost of plan - D * Pi)
    model.setObjective(
        s[0] * initialCost + gp.quicksum( f_i[t]*y[t] + c_i[t]*x[t] + h_i[t]*s[t] for t in range(1, T+1)) #Cost of Plan
        - gp.quicksum(dual_capacity[t]*(r_i*x[t] + tau_i*y[t]) for t in range(1, T+1)) #reduced costs capacity
    , GRB.MINIMIZE)


    # Constraints
    model.addConstr(s[0] == 0, name="no_inventory0") #-> To enforce feasibility
    model.addConstr(s[T] == 0, name="no_inventoryT")
    
    for t in range(1, T+1):
        model.addConstr(s[t-1] + x[t] == demand_i[t] + s[t], name=f"demand_satisfied_{t}_{i}") #demand satisfied 
        model.addConstr(x[t] <= gp.quicksum(demand_i[m] for m in range(t, T+1)) * y[t], name=f"setup_constraint_{t}")

    model.optimize()
    model.update()
    
    if model.objVal - dual_convexity[i]  < -1e-6: 
        #print("--Better Schedule Found--")
        plan = {
            'x': [x[t].X for t in range(T+1)],
            'y': [y[t].X for t in range(T+1)],
            's': [s[t].X for t in range(T+1)],
            'cost': s[0].X * initialCost + sum(f_i[t]*y[t].X + c_i[t]*x[t].X + h_i[t]*s[t].X for t in range(1, T+1)),
            'cap': [r_i*x[t].X + tau_i*y[t].X for t in range(T+1)]
        }
        #print(plan)
        #print('Reduced Cost:', sum(dual_capacity[t]*(r_i*x[t].X + tau_i*y[t].X) for t in range(1, T+1)))
        return plan
    return None

In [41]:
#step 5 column generation

def column_generation():
    columns = initial_columns()
    converged = False
    iteration = 0
    start_time = time.time()

    while not converged:
        iteration += 1
        #print(f"\n=== Iteration {iteration} ===")

        final_obj, duals, z = solve_rmp(columns)
        dual_capacity = duals[:T]
        dual_convexity = duals[T:]

        #print(f"RMP Objective: {final_obj}")
        #print(f"Dual capacity: {dual_capacity}")
        #print(f"Dual convexity: {dual_convexity}")
        
        #print(z)

        new_columns = 0

        for i in range(1, N+1):
            new_plan = solve_subproblem(i, T, demand[i], f[i], c[i], h[i], r[i], tau[i], duals)
            if new_plan:
                columns[i].append(new_plan)
                new_columns += 1

        #print(f"Current Objective Value = {final_obj}")

        # Always show current plans per product
        # print(f"\n--- Current plans per product (only x's) ---")
        # for i in range(1, N+1):
        #     print(f"Product {i}:")
        #     for idx, plan in enumerate(columns[i]):
        #         print(f"  Column {idx}: {plan['x']}")

        if new_columns == 0:
            #print(f"\nNo new columns added. Convergence reached at iteration {iteration}.")
            converged = True

    total_time = time.time() - start_time
    print(f"\nConverged in {iteration} iterations. Final ZDW = {final_obj:.2f}, Time = {total_time:.2f}s")

    # i = 1
    # for k,plan in enumerate(columns[i]):
    #     if z.get((i, k), 0) > 1e-4:
    #             selected = True
    #             for t in range(1, T+1):
    #                 print({
    #                     "Product": i,
    #                     "Period": t,
    #                     "Production": plan['x'][t]
    #                 })
    #             break

    FinalPlans = []
    for i in range(1, N+1):
        for k, plan in enumerate(columns[i]):
            if z[i, k] > 1e-4:
                FinalPlans.append([f"Product {i}", plan])
                break

    return FinalPlans, total_time, iteration, final_obj

In [43]:
# if __name__ == "__main__":
#     FinalPlans = column_generation()

In [45]:
# print(FinalPlans)
# # Initialize a list of rows for the CSV
# columns = ['Period']

# # Add 'x' and 'capacity' columns for each product
# for product in FinalPlans:
#     #print(product)
#     product_name = product[0]
#     #print(product_name)
#     columns.append(f'{product_name}_x')
#     #columns.append(f'{product_name}_cap')
#     columns.append(f'{product_name}_s')

# # Prepare data for each period
# rows = []

# for t in range(1, T+1):
#     row = {'Period': t}
#     #total_capacity_used = 0  # Initialize total capacity used for the current period
#     for product in FinalPlans:
#         product_name = product[0]
#         x_values = product[1]['x']
#         #cap_values = product[1]['cap']
#         s_values = product[1]['s']
#         row[f'{product_name}_x'] = x_values[t]
#         #row[f'{product_name}_cap'] = cap_values[t]
#         row[f'{product_name}_s'] = s_values[t]
#         #total_capacity_used += cap_values[t]  # Sum the capacity for the current period
#     #row['Total_Capacity_Used'] = total_capacity_used  # Add the total capacity used to the row
#     rows.append(row)

# # Convert the data to a DataFrame
# df = pd.DataFrame(rows, columns=columns + ['Total_Capacity_Used'])

# # Save to CSV
# csv_file = 'plan_output_with_total_capacity.csv'
# df.to_csv(csv_file, index=False)

# print(f"CSV file saved as {csv_file}")

In [49]:
sheetNames = ['Data-20-12 (1)', 'Data-20-12 (2)', 'Data-20-24 (1)', 'Data-20-24 (2)', 'Data-100-24 (1)', 'Data-100-24 (2)', 'Data-200-24']
for sheet in sheetNames:
    print(f'Solving sheet: {sheet}')
    N, T, demand, c, h, f, r, kappa, tau, initialCost = defineData(sheet)
    finalPlans,  solveTime, iteration, final_obj = column_generation()
    print('----------')


Solving sheet: Data-20-12 (1)
----------

Converged in 12 iterations. Final ZDW = 89397.10, Time = 1.24s
----------
Solving sheet: Data-20-12 (2)
----------

Converged in 17 iterations. Final ZDW = 14328002.28, Time = 1.23s
----------
Solving sheet: Data-20-24 (1)
----------

Converged in 14 iterations. Final ZDW = 5224857.22, Time = 1.62s
----------
Solving sheet: Data-20-24 (2)
----------

Converged in 23 iterations. Final ZDW = 6705626.82, Time = 2.82s
----------
Solving sheet: Data-100-24 (1)
----------

Converged in 23 iterations. Final ZDW = 36878224.58, Time = 14.73s
----------
Solving sheet: Data-100-24 (2)
----------

Converged in 23 iterations. Final ZDW = 36878224.58, Time = 14.63s
----------
Solving sheet: Data-200-24
----------

Converged in 23 iterations. Final ZDW = 73756449.16, Time = 29.26s
----------
